### Converting Tensorflow Model to TensorFlow Lite Model

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [22]:
!pwd

/root/AI-Projects/Fire_Detection


In [24]:
# Convert the model
saved_model_dir = '/root/AI-Projects/Fire_Detection/final_model/1'
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) # path to the SavedModel directory
tflite_model = converter.convert()

In [25]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

In [ ]:
builder = tfds.ImageFolder('s3://swifty-datasets/firenet_data/')
print(builder.info)  # num examples, labels... are automatically calculated
ds = builder.as_dataset(split='train', shuffle_files=True,  as_supervised = True)
tfds.show_examples(ds, builder.info)

In [ ]:
# Gather results for the randomly sampled test images
predictions = []
test_labels = []
test_images = []

for img, label in ds.take(50):
  interpreter.set_tensor(input_index, img)
  interpreter.invoke()
  predictions.append(interpreter.get_tensor(output_index))
  test_labels.append(label[0])
  test_images.append(np.array(img))

In [ ]:
# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)